Part - 2:
=========

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import DataLoader, TensorDataset
import time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, roc_curve, auc

In [2]:
df = pd.read_csv('dataset.csv')
print(df.describe())

               f3      target
count  766.000000  766.000000
mean    69.118799    0.349869
std     19.376901    0.477240
min      0.000000    0.000000
25%     62.500000    0.000000
50%     72.000000    0.000000
75%     80.000000    1.000000
max    122.000000    1.000000


In [3]:
# we see that there are alphabets in numeric columns so we replace them with mean of that respective column
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col].fillna(df[col].mean(), inplace=True)
df.tail

C:\Users\ruthv\AppData\Local\Temp\ipykernel_9092\399973658.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


<bound method NDFrame.tail of             f1     f2  f3         f4          f5        f6        f7  target
0     6.000000  148.0  72  35.000000    0.000000  33.60000  0.627000       1
1     1.000000   85.0  66  29.000000    0.000000  26.60000  0.351000       0
2     8.000000  183.0  64   0.000000    0.000000  23.30000  0.672000       1
3     1.000000   89.0  66  23.000000   94.000000  28.10000  0.167000       0
4     0.000000  137.0  40  35.000000  168.000000  43.10000  2.288000       1
..         ...    ...  ..        ...         ...       ...       ...     ...
761   9.000000   89.0  62   0.000000    0.000000  22.50000  0.472128       0
762  10.000000  101.0  76  48.000000  180.000000  31.99817  0.171000       0
763   2.000000  122.0  70  27.000000   80.091503  36.80000  0.340000       0
764   3.849673  121.0  72  23.000000  112.000000  26.20000  0.245000       0
765   1.000000  126.0  60  20.542484    0.000000  30.10000  0.349000       1

[766 rows x 8 columns]>

In [4]:
scaler = StandardScaler()
df1 = df.drop('target', axis=1)
column = df1.columns
df1[column] = scaler.fit_transform(df1[column])
df1['target'] = df['target']
df1.tail

<bound method NDFrame.tail of                f1        f2        f3            f4        f5            f6  \
0    6.382139e-01  0.849057  0.148790  9.070150e-01 -0.695096  2.030729e-01   
1   -8.457789e-01 -1.125480 -0.161060  5.305956e-01 -0.695096 -6.843559e-01   
2    1.231811e+00  1.946022 -0.264343 -1.288765e+00 -0.695096 -1.102715e+00   
3   -8.457789e-01 -1.000113 -0.161060  1.541762e-01  0.120709 -4.941926e-01   
4   -1.142577e+00  0.504297 -1.503740  9.070150e-01  0.762938  1.407440e+00   
..            ...       ...       ...           ...       ...           ...   
761  1.528610e+00 -1.000113 -0.367626 -1.288765e+00 -0.695096 -1.204136e+00   
762  1.825408e+00 -0.624010  0.355356  1.722590e+00  0.867083  4.503972e-16   
763 -5.489804e-01  0.034169  0.045507  4.051224e-01  0.000000  6.087546e-01   
764  1.318050e-16  0.002827  0.148790  1.541762e-01  0.276927 -7.350661e-01   
765 -8.457789e-01  0.159536 -0.470909  2.228851e-16 -0.695096 -2.406415e-01   

               f7  ta

In [5]:
X = df.drop('target', axis=1)  # Replace 'target_column' with the actual target column name
Y = df['target']
ros = RandomOverSampler(random_state=42)
X, Y = ros.fit_resample(X, Y)
# Split the data into training and testing sets (80% train, 20% test by default)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.175, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
Y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
print("X_train size: ", X_train_tensor.shape)
print("Y_train size: ", Y_train_tensor.shape)
print("X_val size: ", X_val_tensor.shape)
print("Y_val size: ", Y_val_tensor.shape)
print("X_test size: ", X_test_tensor.shape)
print("Y_test size: ", Y_test_tensor.shape)

X_train size:  torch.Size([697, 7])
Y_train size:  torch.Size([697])
X_val size:  torch.Size([149, 7])
Y_val size:  torch.Size([149])
X_test size:  torch.Size([150, 7])
Y_test size:  torch.Size([150])


Tuning hyperparameters:
=======================
1. Dropout value:
-----------------

In [6]:
class NN_dropout(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(NN_dropout, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [7]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
dropout_rate = [0.3, 0.5, 0.7]
for dropout in dropout_rate:
    model = NN_dropout(input_size, hidden_size, output_size, dropout)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.007)

    num_epochs = 300
    batch_size = 64
    best_val_loss = float('inf')

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            predictions = torch.round(torch.sigmoid(outputs))
            correct_train += (predictions.squeeze(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
    
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

                val_predictions = torch.round(torch.sigmoid(val_outputs))
                correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = correct_val / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if (epoch+1)%10==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
    # Evaluation on the test data
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    test_losses = [test_loss] * num_epochs
    test_accuracies = [test_accuracy] * num_epochs

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Dropout: {dropout}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.6657, Validation Loss: 0.6496, Training Accuracy: 0.6270, Validation Accuracy: 0.6443
Epoch 20/300, Training Loss: 0.6425, Validation Loss: 0.6336, Training Accuracy: 0.6585, Validation Accuracy: 0.6376
Epoch 30/300, Training Loss: 0.6269, Validation Loss: 0.6045, Training Accuracy: 0.6614, Validation Accuracy: 0.6913
Epoch 40/300, Training Loss: 0.6260, Validation Loss: 0.6164, Training Accuracy: 0.6801, Validation Accuracy: 0.6510
Epoch 50/300, Training Loss: 0.5922, Validation Loss: 0.5809, Training Accuracy: 0.7016, Validation Accuracy: 0.7047
Epoch 60/300, Training Loss: 0.6309, Validation Loss: 0.5849, Training Accuracy: 0.6356, Validation Accuracy: 0.7248
Epoch 70/300, Training Loss: 0.5982, Validation Loss: 0.5771, Training Accuracy: 0.6887, Validation Accuracy: 0.7248
Epoch 80/300, Training Loss: 0.5891, Validation Loss: 0.5692, Training Accuracy: 0.6973, Validation Accuracy: 0.7181
Epoch 90/300, Training Loss: 0.5868, Validation Loss: 0.5781, Tr


2. Batch size:
---------------

In [12]:
class NN_batchsize_lr(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN_batchsize_lr, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [15]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.007)
batch_size_list = [32, 64, 128]
for batch in batch_size_list:
    
    num_epochs = 300
    batch_size = batch
    best_val_loss = float('inf')

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            predictions = torch.round(torch.sigmoid(outputs))
            correct_train += (predictions.squeeze(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
    
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

                val_predictions = torch.round(torch.sigmoid(val_outputs))
                correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = correct_val / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if (epoch+1)%10==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
    # Evaluation on the test data
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    test_losses = [test_loss] * num_epochs
    test_accuracies = [test_accuracy] * num_epochs

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Batch size: {batch_size}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.6879, Validation Loss: 0.6778, Training Accuracy: 0.5739, Validation Accuracy: 0.5503
Epoch 20/300, Training Loss: 0.6846, Validation Loss: 0.6624, Training Accuracy: 0.5839, Validation Accuracy: 0.6242
Epoch 30/300, Training Loss: 0.6636, Validation Loss: 0.6515, Training Accuracy: 0.6184, Validation Accuracy: 0.6510
Epoch 40/300, Training Loss: 0.6638, Validation Loss: 0.6640, Training Accuracy: 0.6212, Validation Accuracy: 0.5906
Epoch 50/300, Training Loss: 0.6486, Validation Loss: 0.6538, Training Accuracy: 0.6600, Validation Accuracy: 0.5973
Epoch 60/300, Training Loss: 0.6617, Validation Loss: 0.6182, Training Accuracy: 0.6155, Validation Accuracy: 0.6040
Epoch 70/300, Training Loss: 0.6215, Validation Loss: 0.6111, Training Accuracy: 0.6557, Validation Accuracy: 0.6644
Epoch 80/300, Training Loss: 0.6770, Validation Loss: 0.6199, Training Accuracy: 0.6471, Validation Accuracy: 0.6980
Epoch 90/300, Training Loss: 0.6336, Validation Loss: 0.5973, Tr

3. Learning Rate:
-------------------

In [14]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning_rates = [0.001, 0.007, 0.01]
for learning in learning_rates:
    optimizer = optim.Adam(model.parameters(), lr=learning)
    num_epochs = 300
    batch_size = 64
    best_val_loss = float('inf')

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            predictions = torch.round(torch.sigmoid(outputs))
            correct_train += (predictions.squeeze(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
    
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

                val_predictions = torch.round(torch.sigmoid(val_outputs))
                correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = correct_val / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if (epoch+1)%10==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
    # Evaluation on the test data
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    test_losses = [test_loss] * num_epochs
    test_accuracies = [test_accuracy] * num_epochs

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Learning Rate: {learning}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.7471, Validation Loss: 0.6789, Training Accuracy: 0.5409, Validation Accuracy: 0.5973
Epoch 20/300, Training Loss: 0.7015, Validation Loss: 0.6758, Training Accuracy: 0.5409, Validation Accuracy: 0.6107
Epoch 30/300, Training Loss: 0.6785, Validation Loss: 0.6740, Training Accuracy: 0.5710, Validation Accuracy: 0.6174
Epoch 40/300, Training Loss: 0.6753, Validation Loss: 0.6670, Training Accuracy: 0.5839, Validation Accuracy: 0.6711
Epoch 50/300, Training Loss: 0.6796, Validation Loss: 0.6651, Training Accuracy: 0.5868, Validation Accuracy: 0.6510
Epoch 60/300, Training Loss: 0.6661, Validation Loss: 0.6557, Training Accuracy: 0.6227, Validation Accuracy: 0.6644
Epoch 70/300, Training Loss: 0.6549, Validation Loss: 0.6479, Training Accuracy: 0.6141, Validation Accuracy: 0.6577
Epoch 80/300, Training Loss: 0.6595, Validation Loss: 0.6449, Training Accuracy: 0.6155, Validation Accuracy: 0.6644
Epoch 90/300, Training Loss: 0.6586, Validation Loss: 0.6438, Tr